In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch

In [2]:
REBUILD_DATA = False
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Run on the CPU")
torch.cuda.device_count()

Running on the GPU


1

In [3]:
class DogsVsCats():
    IMG_SIZE = 50
    CATS = "kagglecatsanddogs_3367a/PetImages/Cat/"
    DOGS = "kagglecatsanddogs_3367a/PetImages/Dog/"
    LABELS = {CATS: 0, DOGS: 1}
    
    training_data = [] 
    cat_count = 0
    dog_count = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            
            for f in tqdm(os.listdir(label)): # TQDM is used to show the progress status
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    if label == self.CATS:
                        self.cat_count += 1
                    elif label == self.DOGS:
                        self.dog_count += 1
                except Exception as e:
                    print(str(e))
                    continue
        np.random.shuffle(self.training_data)  # It is in place. So no need to reassign
        np.save("training_data.npy", self.training_data)
        print("Cats:", self.cat_count)
        print("Dogs:", self.dog_count)

In [4]:
if REBUILD_DATA:
    dogsvcats = DogsVsCats()
    dogsvcats.make_training_data()

In [5]:
training_data = np.load("training_data.npy", allow_pickle = True)

In [6]:
print((len(training_data)))

24946


In [7]:
print(training_data[1])

[array([[237, 239, 236, ..., 106, 200, 216],
       [231, 237, 240, ..., 192, 207, 224],
       [239, 239, 237, ..., 203, 228, 203],
       ...,
       [197, 224, 232, ...,  66, 123, 164],
       [192, 197, 197, ..., 101, 171, 172],
       [184, 187, 193, ..., 170, 172, 173]], dtype=uint8)
 array([0., 1.])]


In [8]:
import matplotlib.pyplot as plt

plt.imshow(training_data[1][0], cmap = "gray")
plt.show()

<Figure size 640x480 with 1 Axes>

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) # Input, Output, Kernel
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        x = torch.randn(50,50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2] # Alternative is np.prd(x.shape)
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim =1)

In [10]:
net = Net().to(device)

torch.Size([128, 2, 2])


In [11]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr = 0.001)

loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0

y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)

print(val_size)

2494


In [12]:
val_size

2494

In [18]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [19]:
def fwd_pass(X, y, train = False):
    if train:
        net.zero_grad()
    outputs = net(X)
    matches = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs, y)]
    
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    return acc, loss    

In [28]:
def test(size = 32):
    
    random_start = np.random.randint(len(test_X) - size)
    X, y = test_X[random_start: random_start + size], test_y[random_start: random_start + size]
    with torch.no_grad():
        val_acc, val_loss = fwd_pass(X.view(-1,1, 50, 50).cuda(), y.cuda())
    return val_acc, val_loss
val_acc, val_loss = test()
print(val_acc, val_loss)

torch.Size([128, 2, 2])
0.625 tensor(0.2469, device='cuda:0')


In [ ]:
import time

MODEL_NAME = f"model-{int(time.time())}"

net = Net.cuda()

optimizer = optim.Adam(net.parameters(), lr = 0.001)

loss_function = nn.MSELoss()

print(MODEL_NAME)

def train():
    BATCH_SIZE = 100
    EPOCHS = 8
    with open("model.log", "a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
                batch_X = train_X[i: i+BATCH_SIZE].view(-1, 1, 50, 50).cuda()
                batch_y = train_y[i: i+BATCH_SIZE].cuda()
                
                acc, loss = fwd_pass(batch_X, batch_y, train= True)
                
                for i%50 == 0:
                    val_acc, val_loss = fwd_pass(size)
                    f.write(f"{MODEL_NAME}, {round(time.time(),3)}, {round(float(acc),2)}, {round(float(loss),4)} ")